In [1]:
import gc
import os

import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.profiler import SimpleProfiler
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [2]:
torch.__version__, torch.cuda.is_available()

('1.13.1+cu117', True)

In [3]:
pl.seed_everything(56)

56

In [4]:
columns = ['variantid', 'main_pic_embeddings_resnet_v1', 'name_bert_64']
train_data = pd.read_parquet('./datasets/train_data.parquet', columns=columns).set_index('variantid')
test_data = pd.read_parquet('./datasets/test_data.parquet', columns=columns).set_index('variantid')
train_data

,main_pic_embeddings_resnet_v1,name_bert_64
variantid,,
51195767,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485..."
53565809,"[[1.1471839, -0.665361, 0.7745614, 0.26716197,...","[-0.6575592, 0.6522429, 0.5426037, -0.54347897..."
56763357,"[[-0.90570974, 1.0296293, 1.0769907, 0.27746, ...","[-0.7384308, 0.70784587, 0.3012653, -0.3583719..."
56961772,"[[0.13133773, -0.5577079, 0.32498044, 0.191717...","[-0.44812852, 0.5283565, 0.28981736, -0.506841..."
61054740,"[[0.21696381, 0.10989461, -0.08012986, 0.69186...","[-0.72692573, 0.75206333, 0.37740713, -0.52502..."
...,...,...
820128810,"[[-1.4492652, -0.80129164, -0.12344764, 0.7194...","[-0.8253241, 0.6785133, 0.53978086, -0.4888316..."
821135769,"[[0.012127608, -0.8534423, 0.5415518, -0.44912...","[-0.7413257, 0.46105132, 0.5639801, -0.5462132..."
822095690,"[[0.4248176, -0.15944786, -0.22844064, 0.42768...","[-0.49261805, 0.56726897, 0.7037877, -0.697246..."


In [5]:
train_pairs = pd.read_parquet('./datasets/train_pairs_w_target.parquet')
test_pairs = pd.read_parquet('./datasets/test_pairs_wo_target.parquet')
train_pairs['target'] = train_pairs['target'].astype(int)
train_pairs

,target,variantid1,variantid2
0,0,51197862,51198054
1,1,53062686,536165289
2,1,53602615,587809782
3,1,53888651,89598677
4,0,56930698,551526166
...,...,...,...
306535,0,817327230,822083612
306536,0,817560551,818069912
306537,0,817854719,817857267
306538,0,820036017,820037019


In [6]:
from sklearn.model_selection import train_test_split

train_pairs, val_pairs = train_test_split(train_pairs, test_size=1/3, random_state=56)

In [42]:
class Args:
    batch_size = 128
    
args = Args()

In [43]:
class ItemsDataset(Dataset):
    def __init__(self, pairs, data):
        super().__init__()
        self.pairs = pairs.values
        self.main_pic_embs = data['main_pic_embeddings_resnet_v1']
        self.name_embs = data['name_bert_64']
        self.pairs_len = len(self.pairs)
        
    def __len__(self):
        return self.pairs_len

    def __getitem__(self, idx):
        target, id1, id2 = self.pairs[idx, :]
        return (
            torch.tensor(np.concatenate([self.main_pic_embs[id1][0], self.name_embs[id1]])),
            torch.tensor(np.concatenate([self.main_pic_embs[id2][0], self.name_embs[id2]])),
            target
        )

In [44]:
def get_data_loader(pairs, data, batch_size, drop_last, shuffle):
    dataset = ItemsDataset(pairs, data)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=0,
        drop_last=drop_last,
        shuffle=shuffle,
        pin_memory=True
    )
    return data_loader

In [45]:
def get_loaders(args):
    train_loader = get_data_loader(
        pairs=train_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=True,
        shuffle=True
    )
    
    val_loader = get_data_loader(
        pairs=val_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=False,
        shuffle=False
    )
    return train_loader, val_loader

In [46]:
train_loader, val_loader = get_loaders(args)
len(train_loader), len(val_loader)

(1596, 799)

In [47]:
for x1, x2, target in train_loader:
    print(x1, x2, target)
    break

tensor([[ 0.3512, -0.7226,  0.0562,  ...,  0.5596,  0.4663, -0.6564],
        [-0.0760,  0.1140, -0.1749,  ...,  0.4524,  0.6216, -0.5421],
        [ 0.1295,  0.6638, -1.1495,  ...,  0.3405,  0.5890, -0.1374],
        ...,
        [-0.2727, -0.3946,  0.6830,  ...,  0.6262,  0.6097, -0.7120],
        [ 0.3715, -0.8268,  0.2563,  ...,  0.6903,  0.4710, -0.7594],
        [-0.2465,  0.5650,  0.2164,  ...,  0.7199,  0.1538, -0.2697]]) tensor([[ 3.5116e-01, -7.2256e-01,  5.6164e-02,  ...,  5.9298e-01,
          4.6295e-01, -6.6532e-01],
        [ 1.7391e-02, -4.8241e-04, -7.9498e-03,  ...,  4.5241e-01,
          6.2162e-01, -5.4208e-01],
        [ 2.2914e-01,  5.9452e-01, -1.2918e+00,  ...,  2.4591e-01,
          5.7773e-01, -1.0669e-01],
        ...,
        [-2.4348e-01, -2.8727e-01,  7.1816e-01,  ...,  6.7026e-01,
          6.4032e-01, -6.8571e-01],
        [ 1.6671e-01, -7.4368e-01,  4.9496e-01,  ...,  6.9033e-01,
          4.7099e-01, -7.5944e-01],
        [ 1.4499e-02,  7.9724e-01,  1.

In [48]:
class Net(pl.LightningModule):
    margin = 0.5
    
    def __init__(self):
        super().__init__()
        
        # self.linear = nn.Linear(128 + 64, 256)
        
        features_num = 128 + 64
        # output_size = 256
        # embedding_size = (features_num + output_size) // 2
        embedding_size = 256
        self.neck = nn.Sequential(
            nn.BatchNorm1d(features_num),
            nn.Linear(features_num, embedding_size, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(embedding_size),
            nn.Linear(embedding_size, embedding_size, bias=False),
            nn.BatchNorm1d(embedding_size),
        )
        # self.linear = nn.Linear(embedding_size, output_size)
        
    def forward(self, x):
        # return self.linear(x)
        # return self.linear(self.neck(x))
        return self.neck(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=1e-4, betas=(0.9, 0.999), weight_decay=0.05
        )
        return (
            {
                "optimizer": optimizer,
            },
        )
    
    def training_step(self, batch, batch_idx):
        self.log('step', batch_idx, logger=True, on_epoch=True)
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        # loss = (1 - labels - dists).abs().mean()
        loss = (1 - labels) * torch.pow(dists, 2) + (labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("train_loss", loss, logger=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):        
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        # loss = (1 - labels - dists).abs().mean()
        loss = (1 - labels) * torch.pow(dists, 2) + (labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("val_loss", loss, logger=True, on_epoch=True, prog_bar=True)   
        
        try:
            auc = roc_auc_score(labels.detach().cpu(), dists.detach().cpu())
        except:
            auc = 0
        self.log("val_auc", auc, logger=True, on_epoch=True, prog_bar=True)
        
    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader

In [49]:
model = Net()

In [50]:
import torchinfo

torchinfo.summary(model)

Layer (type:depth-idx)                   Param #
Net                                      --
├─Sequential: 1-1                        --
│    └─BatchNorm1d: 2-1                  384
│    └─Linear: 2-2                       49,152
│    └─ReLU: 2-3                         --
│    └─BatchNorm1d: 2-4                  512
│    └─Linear: 2-5                       65,536
│    └─BatchNorm1d: 2-6                  512
Total params: 116,096
Trainable params: 116,096
Non-trainable params: 0

In [51]:
trainer = pl.Trainer(
    accelerator='gpu', 
    devices=1,
    profiler='advanced',
    precision=32,
    check_val_every_n_epoch=1,
    max_epochs=10
)

trainer.fit(model)

Sanity Checking: 0it [00:00, ?it/s]

2023-05-19 14:40:36,781 [31248] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

2023-05-19 14:40:36,829 [31248] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [52]:
dists, labels = [], []
for x1, x2, target in tqdm(val_loader):
    out1 = model.forward(x1).detach().cpu()
    out2 = model.forward(x2).detach().cpu()
    
    fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
    out1 = out1.div(fnorm.expand_as(out1))
    fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
    out2 = out2.div(fnorm.expand_as(out2))
        
    dists.append(
        nn.PairwiseDistance()(out1, out2).numpy()
    )
    labels.append(target.numpy())
dists = np.concatenate(dists)
labels = np.concatenate(labels)

  0%|          | 0/799 [00:00<?, ?it/s]

In [53]:
dists

array([0.03664988, 0.31131503, 0.13132453, ..., 0.18759945, 0.26426277,
       0.06078308], dtype=float32)

In [54]:
labels

array([0, 1, 0, ..., 1, 0, 1], dtype=int64)

In [55]:
from sklearn.metrics import roc_auc_score

In [56]:
roc_auc_score(labels, dists)

0.8293653754020429

In [57]:
train_dataset = ItemsDataset(train_pairs, train_data)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=args.batch_size,
    num_workers=0,
    drop_last=False,
    shuffle=False,
    pin_memory=True
)

In [58]:
features, labels = [], []
for x1, x2, target in tqdm(train_loader):
    out1 = model.forward(x1).detach().cpu()
    out2 = model.forward(x2).detach().cpu()
    
    fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
    out1 = out1.div(fnorm.expand_as(out1))
    fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
    out2 = out2.div(fnorm.expand_as(out2))
    
    features.append(np.concatenate([out1, out2, nn.PairwiseDistance()(out1, out2).numpy().reshape((-1, 1))], axis=1))
    labels.append(target.numpy())
features = np.concatenate(features)
labels = np.concatenate(labels)    

  0%|          | 0/1597 [00:00<?, ?it/s]

In [59]:
val_features, val_labels = [], []
for x1, x2, target in tqdm(val_loader):
    out1 = model.forward(x1).detach().cpu()
    out2 = model.forward(x2).detach().cpu()
    
    fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
    out1 = out1.div(fnorm.expand_as(out1))
    fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
    out2 = out2.div(fnorm.expand_as(out2))
    
    val_features.append(np.concatenate([out1, out2, nn.PairwiseDistance()(out1, out2).numpy().reshape((-1, 1))], axis=1))
    val_labels.append(target.numpy())
val_features = np.concatenate(val_features)
val_labels = np.concatenate(val_labels)

  0%|          | 0/799 [00:00<?, ?it/s]

In [60]:
from catboost import CatBoostClassifier, Pool, cv

In [61]:
train_pool = Pool(
    data=features,
    label=labels,
)

val_pool = Pool(
    data=val_features,
    label=val_labels,
)

In [62]:
params = {
    'loss_function': 'CrossEntropy',
    'custom_metric': ['AUC'],
    'task_type': 'CPU',
}

In [63]:
model_cb = CatBoostClassifier(**params, random_seed=56)
model_cb.fit(train_pool, eval_set=val_pool, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [64]:
np.max(model_cb.get_evals_result()['validation']['AUC'])

0.846490944059759